# 0 Helper Functions

In [ ]:
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    '''画方框。'''
    # make a copy of the image
    draw_img = np.copy(img)
    # draw each bounding box on your image copy using cv2.rectangle()
    # return the image copy with boxes drawn
    for box in bboxes:
        cv2.rectangle(draw_img,box[0],box[1],color,thick)
    return draw_img # Change this line to return image copy with boxes

In [ ]:
def color_hist(img, color_space = 'RGB', nbins=32, bins_range=(0, 256)):
    '''颜色的 histogram 特征'''
    # Convert image to new color space (if specified)
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)  
    # Compute the histogram of the RGB channels separately
    rhist = np.histogram(feature_image[:,:,0], bins=32, range=(0, 256))
    ghist = np.histogram(feature_image[:,:,1], bins=32, range=(0, 256))
    bhist = np.histogram(feature_image[:,:,2], bins=32, range=(0, 256))
    # Generating bin centers
    bin_edges = rhist[1]
    bin_centers = (bin_edges[1:]  + bin_edges[0:len(bin_edges)-1])/2
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((rhist[0], ghist[0], bhist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return rhist, ghist, bhist, bin_centers, hist_features

In [ ]:
def bin_spatial(img, color_space='RGB', size=(32, 32)):
    '''缩减图片的分辨率'''
    # Convert image to new color space (if specified)
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)    
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(feature_image,size).ravel()
    # Return the feature vector
    return features

In [ ]:
def get_hog_features(img, orient = 9, pix_per_cell = 8, cell_per_block = 2, vis=False, feature_vec=True):
    '''获取 HOG 特征'''
    if vis == True:
        # Use skimage.hog() to get both features and a visualization
        features, hog_image = hog(img,orient,(pix_per_cell,pix_per_cell),(cell_per_block,cell_per_block),visualise = vis, feature_vector= feature_vec) 
        return features, hog_image
    else:      
        # Use skimage.hog() to get features only
        features, _ = hog(img,orient,p(pix_per_cell,pix_per_cell),(cell_per_block,cell_per_block), visualise = vis)
        return features

In [ ]:
from sklearn.preprocessing import StandardScaler

def extract_features(imgs, color_space = 'RGB')
    '''合并特征'''
    features = []
    all_features = []
    for img in imgs:
        hog_features = get_hog_features(img)
        _, _, _, _, color_features = color_hist(img)
        features.append(np.concatenate([hog_features,color_features]))
    # Fit a per-column scaler
    if all_features = []:
        all_features = features
    all_features = np.vstack((all_features,features)).astype(np.float64)
    X_scaler = StandardScaler().fit(all_features)
    # Apply the scaler to X
    scaled_all = X_scaler.transform(all_features)
    return scaled_all

In [ ]:
try:
    from sklearn.cross_validation import train_test_split
except:
    from sklearn.model_selection import train_test_split

def split_data(data):
    # rand_state = np.random.randint(0, 100)
    rand_state = 42
    X_train, X_test, y_train, y_test = train_test_split(
                scaled_X, y, test_size=0.2, random_state=rand_state)
    return X_train, X_test, y_train, y_test

In [ ]:
from sklearn import svm, grid_search
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

def train_model(data,label,parameters = param_grid)
    '''训练模型'''
    svr = svm.SVC()
    clf = grid_search.GridSearchCV(svr, parameters)
    clf.fit(data,label)
    model = clf
    return model

In [ ]:
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    '''移动搜索的窗口'''
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# 1 Histogram of Oriented Gradients (HOG)

## 1.1 Explain how you extracted HOG features from the training images. Explain how you settled on your final choice of HOG parameters.

## 1.2 Describe how you trained a classifier using your selected HOG features (and color features if you used them).

# 2 Sliding Window Search

## 2.1 Describe how you implemented a sliding window search. How did you decide what scales to search and how much to overlap windows?

## 2.2 Show some examples of test images to demonstrate how your pipeline is working. How did you optimize the performance of your classifier?

# 3 Video Implementation

## 3.1 Provide a link to your final video output. 

Your pipeline should perform reasonably well on the entire project video (somewhat wobbly or unstable bounding boxes are ok as long as you are identifying the vehicles most of the time with minimal false positives.)

## 3.2 Describe how (and identify where in your code) you implemented some kind of filter for false positives and some method for combining overlapping bounding boxes.